# Why is Julia fast?

Or rather, how *can* Julia be fast -- since it's perfectly possible to write slow code in Julia.

The point is that if you start with a slow piece of Julia code, it is very possible that by following a few rules, you can tweak it a bit and rapidly end up with a piece of Julia code that is almost identical, but now blindingly fast -- roughly as fast as optimized C! 

How can this be possible? A large part of the reason is through the clever design of the language, and how it handles **types**.

Although it is possible to get a long way in Julia without worrying about, thinking about, or even mentioning types, much of the power of Julia resides in the way it deals with types

## What is a type?

A type is a label that tells the computer how to interpret a block of data in memory. Julia allows us to play around with that to some extent:

In [ ]:
x = 0.1

In [ ]:
bits(x)

In [ ]:
typeof(x)

In [ ]:
y = reinterpret(Int64, x)

In [ ]:
bits(y)

The exact same bit pattern is interpreted in two completely different ways. Indeed, we can even interpret this as a [fixed size] vector of two `Int32`:

In [ ]:
using StaticArrays
reinterpret(SVector{2, Int32}, [x])

## Code specialization

Suppose we define a function in Julia and apply it to values of different types:

In [ ]:
f(x, y) = 2x * y 

In [ ]:
f(3, 4)    

In [ ]:
f(3.5, 4.5)     

As we saw above, the internal representations of `Int64`s and `Float64`s are very different, so Julia must be calling different functions for the multiplication and addition.

Perhaps every time `f` is called, it is doing the analysis (at "run-time"); this is what Python does, for example.

In fact, though, Julia is **much more clever** than this: when you call `f` with a new combination of types, **Julia compiles a new specialized version of `f` for that "type signature"**. The next time that `f` is called with that specific combination of types, Julia just reuses the already-compiled specialized version.

Julia performs **type inference** on the code, in which the input types are propagated through the code, so that Julia can work out what the type of each variable in the function is.

Furthermore, since everything is ["just-in-time" or "ahead-of-time" compiled](https://www.youtube.com/watch?v=7KGZ_9D_DbI), this all happens, in the best case, **at compile time**. 

### Introspection

We can make Julia tell us some of this via its excellent **introspection** capabilities:

In [ ]:
@code_lowered f(3, 4)

In [ ]:
@code_typed f(3, 4)

We see that Julia has been able to infer what the types of each variable are, and what type is returned from this version of the function.

In [ ]:
@code_native f(3.5, 4.5)

Note that `f(3, 4.5)` is a different case, where we are mixing two types.

## Type stability

For the above to work, Julia must be able to successfully complete type inference and find a unique type for each variable in the function, otherwise there is a serious performance penalty. [This is being mitigated in Julia 0.7.]

For example, here is a "classic" example where type inference can fail:

In [ ]:
function sum1(n)
    s = 0
    
    for i in 1:n
        s += i / 2
    end
    
    return s
end

If we just run the function, everything seems to go smoothly:

In [ ]:
sum1(10)

But now we time it:

In [ ]:
@time sum1(10)

There is no reason for this simple function to be allocating memory. What is going on? Let's use introspection:

In [ ]:
@code_warntype sum1(10)

The output of `@code_warntype` is not always very readable, but anything in <font color="red"> red </font> is bad news: it signals a **type instability**, where Julia is unable to determine a unique type for a variable.

Very recently, a new package has come out that gives us a more user-friendly version of this information:

In [ ]:
using Traceur

In [ ]:
@trace sum1(10)

We see that the variable `s` is the culprit: it is first an `Int64` and then a `Float64`. Julia is thus unable to determine which type it should actually be, so that the variable will be **boxed** and leads to **dynamic dispatch**, in which the exact version of the function is determined at run-time. This usually represents the death of good performance (cf. Python, which does exactly this).

In this case, the solution is simple: we initialize `s` as a `Float64` from the start:

In [ ]:
function sum2(n)
    s = 0.0  
    
    for i in 1:n
        s += i / 2
    end 
    
    return s
end

In [ ]:
sum1(1)
@time sum1(10^6)

In [ ]:
sum2(1) 
@time sum2(10^6)   

**Exercise**: To what extent has this effect been mitigated in Julia 0.7? (Note that currently only the REPL works with Julia 0.7; neither IJulia nor Juno work stably yet.)

## Global variables: just don't

This is the reason why global variables are bad news in Julia: their type could change, in principle, at any time, and so they *always* lead to type instability.

Except, that is, if they are declared as `const`. This, confusingly, does not mean that their *values* can't change, but rather that their types can't change.

### Don't use globals; declare constants as `const`

In [ ]:
a = 1
f() = a
 
@code_warntype f() 

In [ ]:
const cc = 1
g() = cc

@code_warntype g()

## Summary

- We **can** talk about types (and should bear them in mind)


- But don't **have to** talk explicitly about types



- Julia **infers** types